## [新浪微博](http://weibo.com)
- 模拟登陆
- 模拟发帖
- 模拟转发
- 模拟关注

### 模拟登陆

In [2]:
#encoding=utf-8
import re
import rsa
import urllib,urllib2
import json
import binascii
import time
import requests

#user,password用户名密码,使用自己注册的sina用户名密码
username = '123456789' #your username
password = '123456789' #your password



#访问网页时，su指向用户名（urlencode后）base64，_后跟时间
session = requests.Session()
su = urllib.quote(username)
su = binascii.b2a_base64(su).strip()
pre_url = 'http://login.sina.com.cn/sso/prelogin.php?entry=weibo&callback=sinaSSOController.preloginCallBack&su='+su+'&rsakt=mod&checkpin=1&client=ssologin.js(v1.4.18)&_='+str((int(time.time()*100)))
login_url = 'http://login.sina.com.cn/sso/login.php?client=ssologin.js(v1.4.18)'

#
req = urllib2.Request(pre_url)
req.add_header('User-Agent','Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116')
req.add_header('Accept','*/*')
req.add_header('Accept-Language','en-US,en;q=0.5')
req.add_header('Accept-Encoding','deflate')
req.add_header('Referer','http://weibo.com/login.php')
response = urllib2.urlopen(req)
jsonText = response.read()

m = re.search('{.*}',jsonText)
if m != None:
    data = m.group()
else:
    print '[-] can not find preloginCallBack Func, please try to check the pre_url!'
    exit()
data = json.loads(data)
servertime = str(data['servertime'])
nonce = data['nonce']
pubkey = data['pubkey']
rsakv = data['rsakv']
time.sleep(10)

#encode username
su = urllib.quote(username)
su = binascii.b2a_base64(su).strip()
#encode password
rsaPublickey = int(pubkey, 16)
key = rsa.PublicKey(rsaPublickey, 65537)
message = str(servertime) + '\t' + str(nonce) + '\n' + str(password)
pwd = binascii.b2a_hex(rsa.encrypt(message, key))

#post data
postdata = {
                    'entry': 'weibo',
                    'gateway': '1',
                    'from': '',
                    'savestate': '7',
                    'pagerefer':'http://login.sina.com.cn/sso/logout.php?entry=miniblog&r=http://weibo.com/logout.php?backurl=/',
                    'userticket': '1',
                    'ssosimplelogin': '1',
                    'vsnf': '1',
                    'vsnval': '',
                    'su': su,
                    'service': 'miniblog',
                    'servertime': servertime,
                    'nonce': nonce,
                    'pwencode': 'rsa2',
                    'sp': pwd,
                    'encoding': 'UTF-8',
                    'url': 'http://weibo.com/ajaxlogin.php?framelogin=1&callback=parent.sinaSSOController.feedBackUrlCallBack',
                    'returntype': 'META',
                    'rsakv' : rsakv,
                    'sr': '1440*900',
                    }
#req = urllib2.Request(login_url)
headers = {}
headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116'
headers['Accept'] = '*/*'
headers['Accept-Language'] = 'en-US,en;q=0.5'
headers['Accept-Encoding'] = 'deflate'
headers['Referer'] = 'http://weibo.com/'
headers['Content-Type'] =  'application/x-www-form-urlencoded'
jsonText = session.post(login_url, postdata, headers = headers)
jsonText.content

'<html>\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=GBK" />\n<title>Sina Passport</title>\n\n\n<script charset="utf-8" src="http://i.sso.sina.com.cn/js/ssologin.js"></script>\n</head>\n<body>\nSigning in ...\n<script>\ntry{sinaSSOController.setCrossDomainUrlList({"retcode":0,"arrURL":["http:\\/\\/crosdom.weicaifu.com\\/sso\\/crosdom?action=login&savestate=1484186433","http:\\/\\/passport.97973.com\\/sso\\/crossdomain?action=login&savestate=1484186433","http:\\/\\/passport.weibo.cn\\/sso\\/crossdomain?action=login&savestate=1"]});}\n\t\tcatch(e){\n\t\t\tvar msg = e.message;\n\t\t\tvar img = new Image();\n\t\t\tvar type = 1;\n\t\t\timg.src = \'http://login.sina.com.cn/sso/debuglog?msg=\' + msg +\'&type=\' + type;\n\t\t}try{sinaSSOController.crossDomainAction(\'login\',function(){location.replace(\'http://weibo.com/ajaxlogin.php?framelogin=1&callback=parent.sinaSSOController.feedBackUrlCallBack&sudaref=weibo.com&ssosavestate=1484186433&ticket=ST-NTY4NzYyNTQ0NQ==-1

In [4]:
#search pages
sea_url = re.search("replace\('(.*)'\)",jsonText.content)
if sea_url is not None:
    sea_url =  sea_url.group().replace('replace(\'','').replace('\')','')
else:
    print '[-] login failed, can not reach sign in pages, please try to avoid identifying code in login process!'
    exit(0)

# visit the search url and get unique id
headers = {}
headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116'
headers['Accept'] = '*/*'
headers['Accept-Language'] = 'en-US,en;q=0.5'
headers['Accept-Encoding'] = 'deflate'
headers['Referer'] = login_url
headers['Content-Type'] =  'application/x-www-form-urlencoded'
jsonText = session.get(sea_url, headers = headers)
content = jsonText.content
#get uid
m = re.search('"uniqueid":"(\d+)"', content)
if m is not None:
    uid =  m.group().split(':')[1].replace('"','')
else:
    print '[-] can not locate your uniqueid, please check if you can locate to arg sea_url!'
    exit(0)
#get cookies
personal_url = 'http://weibo.com/u/'+str(uid)
headers = {}
headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116'
headers['Accept'] = '*/*'
headers['Accept-Language'] = 'en-US,en;q=0.5'
headers['Accept-Encoding'] = 'deflate'
headers['Referer'] = login_url

headers['Content-Type'] =  'application/x-www-form-urlencoded'
jsonText = session.get(personal_url, headers = headers)
jsonText.headers

{'LB_HEADER': 'alice27', 'DPOOL_HEADER': 'alice96', 'Transfer-Encoding': 'chunked', 'Set-Cookie': 'TC-Page-G0=1bbd8b9d418fd852a6ba73de929b3d0c;Path=/', 'Expires': 'Sat, 26 Jul 1997 05:00:00 GMT', 'Vary': 'Accept-Encoding', 'Server': 'WeiBo', 'Connection': 'close', 'Pragma': 'no-cache', 'Cache-Control': 'no-cache, must-revalidate', 'Date': 'Wed, 13 Jan 2016 02:00:52 GMT', 'Content-Type': 'text/html; charset=utf-8'}

### 发微博

In [23]:
def add_new(content ,resp):
    addurl = "http://weibo.com/aj/mblog/add?ajwvr=6&__rnd" + str(int(time.time()*1000))
    data = {'text':content, 'rank':'0', 'rankid':'', 'location':'v6_content_home', 'module':'stissue', '_t':'0','appkey':'',
           'tyle_type':'1', 'pic_id':'', 'pdetail':'', 'pub_source':'main_', 'pub_type':'dialog' }
    headers = {}
    headers['set-cookie'] = resp.headers['set-cookie']
    headers['Referer'] = 'http://weibo.com/u/'+str(uid)+'/home'
    respon = session.post(addurl, data, headers=headers)
    if respon.status_code == 200:
        print '[+] post success!'
    else:
        print '[-] post failed!'
        exit(1)
add_new("I'm a test", jsonText)

[+] post success!
{"code":"100000","msg":"","data":{"html":"\t\t \n\t\t\t\t\t\t    \t    \t\t    \t    \t    \t    \t    \t\t    \t<div   tbinfo=\"ouid=5687625445\"  diss-data=\"group_source=\" class=\"WB_cardwrap WB_feed_type S_bg2\"  mid=\"3930833410655884\"  action-type=\"feed_list_item\"  >\n         <div class=\"WB_feed_detail clearfix\" node-type=\"feed_content\" \n    \t>\n    \t                <div class=\"WB_screen W_fr\">\n            <div class=\"screen_box\">\n                <a href=\"javascript:void(0);\" action-type=\"fl_menu\"><i class=\"W_ficon ficon_arrow_down S_ficon\">c<\/i><\/a>\r\n<div class=\"layer_menu_list\" node-type=\"fl_menu_right\" style=\"display:none; position: absolute; z-index: 999;\">\r\n    <ul>\r\n                        <li><a href=\"javascript:;\" action-type=\"feed_list_delete\" title=\"\u5220\u9664\">\u5220\u9664<\/a><\/li>\r\n                \t<li><a href=\"javascript:;\" action-type=\"fl_friendVisible\" action-data=\"visible=2\" title=\"\u8f6c\

### 自动转发

In [31]:
#
def forward(mid, content, domain): 
    forwardurl = 'http://d.weibo.com/aj/v6/mblog/forward?ajwvr=6&domain='+str(domain)+'&__rnd=' + str(int(time.time()*1000))
    data = {
        'pic_src':'',
        'pic_id':'',
        'appkey':'',
        'mid':str(mid), 
        'style-type':1,
        'mark':'',
        'reason':content, 
        'location':'page_'+str(domain)+'_home',
        'pdetail':str(domain),
        'module':'',
        'page_module_id':'',
        'refer_sort':'',
        'rank':0,
        'rankid':'',
        'filter_actionlog':'102803_ctg1_99991_-_ctg1_99991',
        '_t':0,
    }
    headers = {}
    headers['set-cookie'] = jsonText.headers['set-cookie']
    headers['Referer'] = 'http://d.weibo.com/'
    respon = session.post(forwardurl, data, headers=headers)
    if respon.status_code == 200:
        print '[+] post success!'
        print respon.content
    else:
        print '[-] post failed!'
        exit(1)
forward(3930416564614210, "zhuanfa", 102803)

[+] post success!
{"code":"100000","msg":"","data":{"html":"\t\t \n\t\t\t\t\t\t    \t    \t\t    \t    \t    \t    \t    \t\t    \t<div  minfo=\"ru=3229125510&rm=3930416564614210\" tbinfo=\"ouid=5687625445&rouid=3229125510\"  diss-data=\"group_source=\" class=\"WB_cardwrap WB_feed_type S_bg2\"  mid=\"3930845444299810\" isForward=\"1\" omid=\"3930416564614210\"  action-type=\"feed_list_item\"  >\n         <div class=\"WB_feed_detail clearfix\" node-type=\"feed_content\" \n    \t>\n    \t                <div class=\"WB_screen W_fr\">\n            <div class=\"screen_box\">\n                <a href=\"javascript:void(0);\" action-type=\"fl_menu\"><i class=\"W_ficon ficon_arrow_down S_ficon\">c<\/i><\/a>\r\n<div class=\"layer_menu_list\" node-type=\"fl_menu_right\" style=\"display:none; position: absolute; z-index: 999;\">\r\n    <ul>\r\n                        <li><a href=\"javascript:;\" action-type=\"feed_list_delete\" title=\"\u5220\u9664\">\u5220\u9664<\/a><\/li>\r\n                \t<

### 自动关注

In [17]:
#dstuid代表名字，fname代表关注人的名字，oid是登陆时获取的你的账号id，resp用来提供cookie
#/gaoyuanyuan?is_hot=1，爬虫需要注意昵称
def followed(dstuid,fname,oid,resp):
        followedurl = "http://weibo.com/aj/f/followed?ajwvr=6&__rnd=" + str(int(time.time()*1000))
        data = {'uid':dstuid,
                'objectid':'',
                'f':'1',
                'extra':'',
                'refer_sort':'',
                'refer_flag':'',
                'location':'page_100505_home',#通过
                'oid':dstuid,
                'wforce':'1',
                'nogroup':'false',
                'fnick':fname,
                '_t':'0'
                }
        headers = {}
        headers['set-cookie'] = resp.headers['set-cookie']
        headers['Referer'] = 'http://weibo.com/u/'+dstuid+'?is_hot=1'
        respon = session.post(followedurl, data, headers=headers)
        if respon.status_code == 200:
        print '[+] post success!'
    else:
        print '[-] post failed!'
        exit(1)
followed('2182038504','我的前任是极品',uid,jsonText)

200
